### Expressivity

In [1]:
import numpy as np
from ansatz import Ansatz
from utils import pennylane_to_qiskit
import pennylane as qml

n_qubits = 5
results = {}
depth = 1
circuits = {}
for ansatz_id in range(1, 20):
    results[ansatz_id] = {}
    super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
    ansatz = super_ansatz.get_circuit()
    param_shape = super_ansatz.get_params_shape()
    if param_shape[1] is not None:
        params1 = np.random.uniform(0, 2 * np.pi, param_shape[0])
        params2 = np.random.uniform(0, 2 * np.pi, param_shape[1])
        params = [params1, params2]

        @qml.qnode(qml.device("default.qubit", wires=n_qubits))
        def circuit(params1, params2):
            ansatz(params1, params2)
            return qml.expval(qml.PauliZ(0))
    else:
        params1 = np.random.uniform(0, 2 * np.pi, param_shape[0])
        params = [params1]
        @qml.qnode(qml.device("default.qubit", wires=n_qubits))
        def circuit(params1):
            ansatz(params1)
            return qml.expval(qml.PauliZ(0))

    qiskit_circuit = pennylane_to_qiskit(circuit, n_qubits, params=params)
    circuits[ansatz_id] = qiskit_circuit

    # Draw the circuit in file
    #filename = f"figures\\ansatz_{ansatz_id}_depth_{depth}.png"
    #qiskit_circuit.draw(output="mpl", filename=filename)

In [2]:
from qleet.analyzers.expressibility import Expressibility
from qleet.interface.circuit import CircuitDescriptor

for ansatz_id in range(1, 20):
    qiskit_circuit = circuits[ansatz_id]
    params = qiskit_circuit.parameters
    circuit_descriptor = CircuitDescriptor(qiskit_circuit, params)
    exp = Expressibility(circuit_descriptor, n_qubits, shots=10000)
    results[ansatz_id]["expressivity"] = exp.expressibility()
    print(f"Ansatz {ansatz_id}: {results[ansatz_id]['expressivity']}")

ModuleNotFoundError: No module named 'tensorflow'